<h1>Problema de Transbordo<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

Iniciamos llamndo a PULP

In [1]:
from pulp import *

Definimos en dos listas los nodos en nuestro problema de transporte:

In [2]:
#Plantas de producción
Plantas=["Denver","Atlanta"]

#Almacenes Distribución
Almacenes=["Kansas","Louisville"]

#Tiendas (Llegada)
Tiendas=["Detroit","Miami","Dallas","Nueva Orleans"]

Producción

In [5]:
produccion={"Denver":600,"Atlanta":400}
produccion

{'Denver': 600, 'Atlanta': 400}

Demanda


In [6]:
demanda={"Detroit":200,"Miami":150,"Dallas":350,"Nueva Orleans":300}
demanda

{'Detroit': 200, 'Miami': 150, 'Dallas': 350, 'Nueva Orleans': 300}

Costos unitarios de transporte de plantas a almacenes:

In [9]:
costos_Planta_Almacen=[ 
    [2,3],
    [3,1]
]
costos_Planta_Almacen=makeDict([Plantas,Almacenes],costos_Planta_Almacen,0)
costos_Planta_Almacen

defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
            {'Denver': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {'Kansas': 2, 'Louisville': 3}),
             'Atlanta': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {'Kansas': 3, 'Louisville': 1})})

Costos unitarios de transporte de almacenes a tiendas:

In [10]:
costos_Almacen_Tiendas=[ 
    [2,6,3,6],
    [4,4,6,5]
]
costos_Almacen_Tiendas=makeDict([Almacenes,Tiendas],costos_Almacen_Tiendas,0)
costos_Almacen_Tiendas

defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
            {'Kansas': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {'Detroit': 2,
                          'Miami': 6,
                          'Dallas': 3,
                          'Nueva Orleans': 6}),
             'Louisville': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {'Detroit': 4,
                          'Miami': 4,
                          'Dallas': 6,
                          'Nueva Orleans': 5})})

Ahora definimos el problema de Programación lineal asociado a este problema de transporte:

In [11]:
probTransbordo=LpProblem("Problema_de_transbordo",LpMinimize)

In [12]:
Rutas_Planta_Almacen= [(p,c) for p in Plantas for c in Almacenes]
Rutas_Planta_Almacen

[('Denver', 'Kansas'),
 ('Denver', 'Louisville'),
 ('Atlanta', 'Kansas'),
 ('Atlanta', 'Louisville')]

In [13]:
Rutas_Almacen_Tiendas= [(p,c) for p in Almacenes for c in Tiendas]
Rutas_Almacen_Tiendas

[('Kansas', 'Detroit'),
 ('Kansas', 'Miami'),
 ('Kansas', 'Dallas'),
 ('Kansas', 'Nueva Orleans'),
 ('Louisville', 'Detroit'),
 ('Louisville', 'Miami'),
 ('Louisville', 'Dallas'),
 ('Louisville', 'Nueva Orleans')]

Definimos las variables

In [14]:
variables_Planta_Almacen=LpVariable.dicts("Ruta",(Plantas,Almacenes),0,None,LpContinuous)
variables_Planta_Almacen

{'Denver': {'Kansas': Ruta_Denver_Kansas,
  'Louisville': Ruta_Denver_Louisville},
 'Atlanta': {'Kansas': Ruta_Atlanta_Kansas,
  'Louisville': Ruta_Atlanta_Louisville}}

In [15]:
variables_Almacen_Tienda=LpVariable.dicts("Ruta",(Almacenes,Tiendas),0,None,LpContinuous)
variables_Almacen_Tienda

{'Kansas': {'Detroit': Ruta_Kansas_Detroit,
  'Miami': Ruta_Kansas_Miami,
  'Dallas': Ruta_Kansas_Dallas,
  'Nueva Orleans': Ruta_Kansas_Nueva_Orleans},
 'Louisville': {'Detroit': Ruta_Louisville_Detroit,
  'Miami': Ruta_Louisville_Miami,
  'Dallas': Ruta_Louisville_Dallas,
  'Nueva Orleans': Ruta_Louisville_Nueva_Orleans}}

Definimos la función objetivo

In [16]:
v1=[variables_Planta_Almacen[p][c]*costos_Planta_Almacen[p][c] for (p,c) in Rutas_Planta_Almacen]
v2=[variables_Almacen_Tienda[p][c]*costos_Almacen_Tiendas[p][c] for (p,c) in Rutas_Almacen_Tiendas]
print(v1+v2)

[2*Ruta_Denver_Kansas + 0, 3*Ruta_Denver_Louisville + 0, 3*Ruta_Atlanta_Kansas + 0, 1*Ruta_Atlanta_Louisville + 0, 2*Ruta_Kansas_Detroit + 0, 6*Ruta_Kansas_Miami + 0, 3*Ruta_Kansas_Dallas + 0, 6*Ruta_Kansas_Nueva_Orleans + 0, 4*Ruta_Louisville_Detroit + 0, 4*Ruta_Louisville_Miami + 0, 6*Ruta_Louisville_Dallas + 0, 5*Ruta_Louisville_Nueva_Orleans + 0]


In [17]:
probTransbordo+= lpSum(v1+v2)

Definimos las restricciones:

In [18]:
for p in Plantas:
    probTransbordo+=lpSum([variables_Planta_Almacen[p][c] for c in Almacenes])<=produccion[p],"Producción_en_%s"%p

In [19]:
for c in Tiendas:
    probTransbordo +=lpSum([variables_Almacen_Tienda[p][c] for p in Almacenes])>=demanda[c],"Demanda_en_%s"%c

In [20]:
for c in Almacenes:
    probTransbordo +=lpSum([variables_Planta_Almacen[p][c] for p in Plantas])==lpSum([variables_Almacen_Tienda[c][t] for t in Tiendas]),"Distribución_en_%s"%c

In [21]:
probTransbordo.writeLP

<bound method LpProblem.writeLP of Problema_de_transbordo:
MINIMIZE
3*Ruta_Atlanta_Kansas + 1*Ruta_Atlanta_Louisville + 2*Ruta_Denver_Kansas + 3*Ruta_Denver_Louisville + 3*Ruta_Kansas_Dallas + 2*Ruta_Kansas_Detroit + 6*Ruta_Kansas_Miami + 6*Ruta_Kansas_Nueva_Orleans + 6*Ruta_Louisville_Dallas + 4*Ruta_Louisville_Detroit + 4*Ruta_Louisville_Miami + 5*Ruta_Louisville_Nueva_Orleans + 0
SUBJECT TO
Producción_en_Denver: Ruta_Denver_Kansas + Ruta_Denver_Louisville <= 600

Producción_en_Atlanta: Ruta_Atlanta_Kansas + Ruta_Atlanta_Louisville <= 400

Demanda_en_Detroit: Ruta_Kansas_Detroit + Ruta_Louisville_Detroit >= 200

Demanda_en_Miami: Ruta_Kansas_Miami + Ruta_Louisville_Miami >= 150

Demanda_en_Dallas: Ruta_Kansas_Dallas + Ruta_Louisville_Dallas >= 350

Demanda_en_Nueva_Orleans: Ruta_Kansas_Nueva_Orleans
 + Ruta_Louisville_Nueva_Orleans >= 300

Distribución_en_Kansas: Ruta_Atlanta_Kansas + Ruta_Denver_Kansas
 - Ruta_Kansas_Dallas - Ruta_Kansas_Detroit - Ruta_Kansas_Miami
 - Ruta_Kansas_Nu

In [22]:
probTransbordo.solve()

1

In [23]:
for v in probTransbordo.variables():
    print(v.name, "=", v.varValue)

Ruta_Atlanta_Kansas = 0.0
Ruta_Atlanta_Louisville = 400.0
Ruta_Denver_Kansas = 550.0
Ruta_Denver_Louisville = 50.0
Ruta_Kansas_Dallas = 350.0
Ruta_Kansas_Detroit = 200.0
Ruta_Kansas_Miami = 0.0
Ruta_Kansas_Nueva_Orleans = 0.0
Ruta_Louisville_Dallas = 0.0
Ruta_Louisville_Detroit = 0.0
Ruta_Louisville_Miami = 150.0
Ruta_Louisville_Nueva_Orleans = 300.0


In [24]:
print("Costominimo= ", value(probTransbordo.objective))

Costominimo=  5200.0
